# OSRM Table Service – Travel Time Matrix Analysis

**Objective:**  
This notebook demonstrates how to use the OSRM *Table* service
to compute travel duration (or distance) matrices between
multiple geographic coordinates.

## Table of Contents

- [Introduction to the Table Service](#introduction-to-the-table-service)
- [Setup](#setup)
- [Using the Table Service](#using-the-table-service)
- [Using the Service](#using-the-service)
  - [General Options](#general-options)
  - [General Parameters](#general-parameters)
  - [Making the Request](#making-the-request)
  - [No Source & No Destination](#no-source--no-destination)
- [With Source & No Destination](#with-source--no-destination)
- [No Source & With Destination](#no-source--with-destination)
- [With Source & With Destination](#with-source--with-destination)
- [Conclusion](#conclusion)

## Introduction to the Table Service

The Table service computes a matrix of travel durations
(or distances) between multiple coordinates.

Unlike the Route service, which returns a single path,
the Table service evaluates all pairwise combinations
and returns a structured duration matrix.

## Setup

We define multiple geographic coordinates
to construct a travel-time matrix.

These points will be visualized on the map
to provide spatial context before computing
the duration matrix.

## Using the Table Service

The Table endpoint follows this structure:

/table/{version}/{profile}/{coordinates}

It returns a duration matrix where each cell
(i, j) represents the travel time from
coordinate i to coordinate j.

In [ ]:
import folium

from folium import Map
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float


def get_folium_map(center_point: Point, points: list[Point], zoom_level: int = 14) -> Map:
    folium_map = folium.Map(
        location=[center_point.latitude, center_point.longitude], zoom_start=zoom_level)

    for point in points:
        folium.Marker(location=[point.latitude, point.longitude],
                      popup='Point').add_to(folium_map)

    return folium_map


point_1 = Point(latitude=53.091631767437555, longitude=8.80205658376942)
point_2 = Point(latitude=53.096631294014614, longitude=8.805747303373913)
point_3 = Point(latitude=53.093590414069496, longitude=8.813944134123425)
center_point = Point(latitude=53.09395115850722, longitude=8.807249340422253)

folium_map = get_folium_map(center_point, [point_1, point_2, point_3])
folium_map

## Using the Service

The Table service allows us to compute
travel durations between multiple locations
in a single request.

Instead of returning a single route,
it produces a structured matrix that
represents the travel time between
each pair of supplied coordinates.

### General Options

All OSRM services follow a consistent
parameter structure.

The table below outlines the core parameters
used when interacting with the Table endpoint.

### General Parameters

| Parameter     | Description |
|--------------|-------------|
| **service**  | Specifies the OSRM operation to execute. Possible values include `route`, `nearest`, `table`, `match`, `trip`, and `tile`. |
| **version**  | Defines the API version used by the service. For OSRM 5.x installations, this is typically `v1`. |
| **profile**  | Determines the routing mode used when generating the graph (e.g., `car`, `bike`, or `foot`). This is defined by the Lua profile used during `osrm-extract`. |
| **coordinates** | A sequence of coordinate pairs formatted as `{longitude},{latitude};{longitude},{latitude};...` or provided as an encoded `polyline`. |
| **format**   | Specifies the output format. Currently, `json` is supported and is used by default. |

### Making the Request

The Table endpoint follows this pattern:

/table/v1/{profile}/{coordinates}?sources={...}&destinations={...}

If no `sources` or `destinations` are specified,
OSRM computes the complete pairwise duration matrix
for all provided coordinates.

### No Source & No Destination

In this configuration, we do not restrict
the origins or destinations.

As a result, the service evaluates every
possible origin–destination combination
among the supplied coordinates.

This produces a full duration matrix.

In [ ]:
service = 'table'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

In [ ]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}"

response = requests.get(url)
response.status_code

A `200` status response confirms that
the request was processed successfully.

Since no filtering parameters were applied,
the returned matrix contains travel durations
between all coordinates in both directions.

In [ ]:
data = response.json()
data.keys()

When neither `sources` nor `destinations` are specified,
the Table service automatically computes travel durations
for every possible origin–destination pair.

For example, if three coordinates are supplied,
the result is a 3 × 3 duration matrix,
representing all nine combinations.

In [ ]:
print(data["durations"])

The `durations` field contains the travel-time matrix.

Each row corresponds to a source location,
while each column represents a destination location.

The value stored at position (i, j)
indicates the travel time in seconds
from source *i* to destination *j*.

When the source and destination are identical,
the duration is naturally zero.

The table below represents the duration matrix
computed by the OSRM Table service.

| Source \ Destination | Point 1 | Point 2 | Point 3 |
|----------------------|---------|---------|---------|
| **Point 1** | 0 | 175.6 | 210.1 |
| **Point 2** | 168.8 | 0 | 193.7 |
| **Point 3** | 191.2 | 171.2 | 0 |

## With Source & No Destination

We can refine the computation by specifying
the `sources` parameter.

This restricts the matrix to use only the selected
coordinate(s) as origin points,
while the remaining coordinates are treated as destinations.

In [ ]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?sources=0"

response = requests.get(url)
response.status_code

The `200` status confirms that the request
was executed successfully with the applied filter.

In this case, we explicitly selected
the first coordinate as the source.

In [ ]:
data = response.json()
data["durations"]

By defining a single source index,
the resulting output is reduced to a 1 × 3 matrix.

This means travel durations are calculated
from the selected source point
to each of the available destination points.

Instead of fixing the origin, we can specify the
`destinations` parameter.

In this example, we set `destinations=0`,
which forces the engine to compute travel durations
from all supplied points **to Point 1 only**.

The response now returns a single-column matrix.

| Source \ Destination | Point 1 | Point 2 | Point 3 |
|----------------------|---------|---------|---------|
| **Point 1** | 0 | 175.6 | 210.1 |

## No Source & With Destination

Instead of fixing the origin, we can specify the
`destinations` parameter.

In this example, we set `destinations=0`,
which forces the engine to compute travel durations
from all supplied points **to Point 1 only**.

The response now returns a single-column matrix.

In [ ]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?destinations=0"

response = requests.get(url)
response.status_code

The `200` status code confirms that the request
was successfully processed with the destination filter applied.

The matrix represents the travel duration **to Point 1 (destination)**
from each of the three source points.

| Source \ Destination | Point 1 |
|----------------------|---------|
| **Point 1** | 0 |
| **Point 2** | 168.8 |
| **Point 3** | 191.2 |

## With Source & With Destination

Finally, we can combine both `sources`
and `destinations` parameters.

This allows us to compute travel durations
between selected origin points
and selected destination points only.

The resulting matrix will include only
the specified origin–destination pairs,
producing a smaller and more focused output.

In [ ]:
import requests

points = [point_1, point_2, point_3]
coordinates = ';'.join(
    [f'{point.longitude},{point.latitude}' for point in points])

url = f"{host}/{service}/{version}/{profile}/{coordinates}?sources=0&destinations=2"

response = requests.get(url)
response.status_code

In [ ]:
data = response.json()
data["durations"]

Since we specified both a source and a destination,
the table service returns a single travel duration.

Here, the duration from **Point 1** to **Point 3**
is **210.1 seconds**, represented as `[[210.1]]`.

## Conclusion

In this notebook, we explored the OSRM **Table** service  
and its role in computing travel-time matrices  
between multiple geographic coordinates.

We demonstrated:

- Generating a complete N × N duration matrix between all supplied points  
- Understanding how rows represent **sources** and columns represent **destinations**  
- Restricting computations using the `sources` parameter  
- Restricting computations using the `destinations` parameter  
- Combining both parameters to extract a single targeted travel duration  

The Table service is particularly useful in routing scenarios  
where pairwise travel analysis is required, such as  
logistics planning, delivery optimization, and fleet coordination.

Since the service operates on graph-based road networks,  
it enables efficient large-scale travel-time computation  
without reconstructing full route geometries.

By leveraging full matrices, filtered sources,  
and controlled destinations, we established a structured  
and scalable approach to travel-time analysis.

This foundation supports downstream tasks such as  
vehicle routing problems (VRP), dispatch optimization,  
and intelligent transportation analytics.